# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [12]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [17]:
# 1) Seu código aqui
url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio"

r = requests.get(url)

data_json = r.json()

df = pd.DataFrame(data_json['items'])

In [18]:
# 2) Seu código aqui
df['status'].value_counts()

#Deferido                                                 105
#Arquivado                                                 53
#Regularizado                                              26

status
Deferido                                                 105
Arquivado                                                 53
Regularizado                                              26
Arquivado por decurso de prazo                            24
Deferido (PVL-IF)                                         24
Encaminhado à PGFN com manifestação técnica favorável     14
Arquivado a pedido                                         7
Em retificação pelo interessado                            3
Arquivado a pedido (PVL-IF)                                1
Arquivado pela STN                                         1
Em análise                                                 1
Name: count, dtype: int64

In [30]:
# 3) Seu código aqui
df['ano_status'] = df['data_status'].str.slice(6, 11)


In [29]:
# 4) Seu código aqui
df['ano_status'].value_counts()


ano_status
2008    37
2007    29
2023    29
2014    20
2010    19
2019    16
2020    15
2013    15
2012    13
2022    11
2011    10
2009     9
2015     8
2017     5
2021     5
2002     4
2016     4
2018     4
2006     4
2003     1
2004     1
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [43]:
#1) Seu código aqui
import pandas as pd
import requests

#Criando chaves para automatizar a consulta
def link_api(estado, interesse):
    
    interesse_municipio ='Munic%C3%ADpio'
    interesse_estado ='Estado'
    base = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=colocaruf&tipo_interessado='

    #montando a URL
    novo_link = base.replace('colocaruf', estado)
    
    if interesse == "M":
        novo_link_com_interesse = novo_link + interesse_municipio
    elif interesse == "E":
        novo_link_com_interesse = novo_link + interesse_estado
        
    r = requests.get(novo_link_com_interesse)

    data_json = r.json()

    df = pd.DataFrame(data_json['items'])
    
    return df


#print(link_api('MG','M'))

In [48]:
# 2) Seu código aqui
nova_base_mg = pd.DataFrame(link_api('MG','E'))

nova_base_mg['status'].value_counts()
#Arquivado por decurso de prazo                            1

status
Encaminhado à PGFN com manifestação técnica favorável    20
Deferido                                                  7
Arquivado a pedido                                        5
Arquivado                                                 3
Arquivado pela STN                                        3
Arquivado por decurso de prazo                            1
Indeferido                                                1
Name: count, dtype: int64

In [62]:
# 3) Seu código aqui
nova_base_ba = pd.DataFrame(link_api('BA','M'))

df_deferido = nova_base_ba[nova_base_ba['status'] == 'Deferido']

df_deferido['interessado'].value_counts()
#contagem_por_municipio = df_deferido.groupby('interessado').size().reset_index(name='contagem')

interessado
Luís Eduardo Magalhães    16
Vitória da Conquista      12
Barreiras                 10
Camaçari                  10
Lauro de Freitas          10
                          ..
Malhada                    1
Sobradinho                 1
Cafarnaum                  1
Itapicuru                  1
Itagibá                    1
Name: count, Length: 185, dtype: int64

In [63]:
# 4) Seu código aqui
nova_base_ba_e = pd.DataFrame(link_api('BA','E'))

nova_base_ba_e.to_csv('base_bahia_estado.csv',index=False)
